<a href="https://colab.research.google.com/github/jcmachicao/deep_learning_2025_curso/blob/main/S05__rnn_demo_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# ------------------------------
# 1. Configuración e hipótesis de dataset
# ------------------------------
BATCH_SIZE = 32     # oraciones por lote
SEQ_LEN = 20        # palabras por oración
INPUT_SIZE = 300    # dimensión del embedding de palabra
HIDDEN_SIZE = 128   # dimensión del estado oculto

# Dataset sintético: entradas aleatorias y etiquetas binarias
X_train = torch.rand(BATCH_SIZE, SEQ_LEN, INPUT_SIZE)
y_train = torch.randint(0, 2, (BATCH_SIZE, 1)).float()

In [8]:
X_train

tensor([[[0.2444, 0.4696, 0.4917,  ..., 0.5005, 0.1271, 0.1840],
         [0.8907, 0.6179, 0.6513,  ..., 0.4863, 0.7914, 0.2613],
         [0.8244, 0.5600, 0.8050,  ..., 0.5694, 0.8992, 0.3049],
         ...,
         [0.8863, 0.4711, 0.9189,  ..., 0.9883, 0.0672, 0.5517],
         [0.3041, 0.7957, 0.3291,  ..., 0.3002, 0.4577, 0.5342],
         [0.2343, 0.6840, 0.5252,  ..., 0.2029, 0.9880, 0.8061]],

        [[0.7918, 0.0627, 0.1028,  ..., 0.9852, 0.2044, 0.1458],
         [0.2438, 0.9858, 0.9834,  ..., 0.7445, 0.9789, 0.3926],
         [0.1837, 0.8360, 0.2065,  ..., 0.2938, 0.0428, 0.0468],
         ...,
         [0.4109, 0.0639, 0.4478,  ..., 0.3273, 0.1054, 0.4674],
         [0.8653, 0.2872, 0.4504,  ..., 0.1453, 0.4862, 0.7763],
         [0.5806, 0.6515, 0.6683,  ..., 0.1594, 0.6664, 0.6364]],

        [[0.7259, 0.0468, 0.6096,  ..., 0.2142, 0.1511, 0.4817],
         [0.3426, 0.2782, 0.1296,  ..., 0.3837, 0.9668, 0.1577],
         [0.6404, 0.4653, 0.1802,  ..., 0.6967, 0.5708, 0.

In [3]:
# ------------------------------
# 2. Definir modelo RNN simple
# ------------------------------
class SimpleRNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes=1):
        super(SimpleRNNClassifier, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, h_n = self.rnn(x)  # out: (batch, seq_len, hidden_size)
        last_hidden = h_n[-1]   # (batch, hidden_size)
        logits = self.fc(last_hidden)
        return self.sigmoid(logits)

# Instanciar modelo
model = SimpleRNNClassifier(INPUT_SIZE, HIDDEN_SIZE)

In [4]:
# ------------------------------
# 3. Entrenamiento básico
# ------------------------------
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 5
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 0.6905
Epoch [2/5], Loss: 0.6715
Epoch [3/5], Loss: 0.6277
Epoch [4/5], Loss: 0.6033
Epoch [5/5], Loss: 0.5776


In [5]:
# ------------------------------
# 4. Guardar pesos
# ------------------------------
torch.save(model.state_dict(), "rnn_weights.pth")

# ------------------------------
# 5. Cargar modelo
# ------------------------------
model_loaded = SimpleRNNClassifier(INPUT_SIZE, HIDDEN_SIZE)
model_loaded.load_state_dict(torch.load("rnn_weights.pth"))
model_loaded.eval()

SimpleRNNClassifier(
  (rnn): RNN(300, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [6]:
# ------------------------------
# 6. Inferencia con nueva data
# ------------------------------
new_data = torch.rand(2, SEQ_LEN, INPUT_SIZE)  # batch de 2 oraciones
with torch.no_grad():
    predictions = model_loaded(new_data)

print("Predicciones para nueva data:", predictions)

Predicciones para nueva data: tensor([[0.4805],
        [0.5352]])
